<a href="https://colab.research.google.com/github/4dh/AppRegressionHW/blob/master/RegressionHW10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Danielle Heymann

CSCI 688: Linear Regression

> HW 10



11.8 Employee salaries. A group of high-technology companies agreed to share employee salary information in an effort to establish salary ranges for technical positions in research development. Data obtained for each employee included current salary (Y), a coded variable indicating highest academic degree obtained (1 = bachelor’s degree, 2 = master’s degree, 3 = doctoral degree), years of experience since last degree (X3), and the number of persons currently supervised (X4). 


> a) Create two indicator variables for the highest degree attained:


Degree	    X1	 X2

Bachelor’s	0	   0

Master’s	  1	   0

Doctoral  	0	   1


In [ ]:
#initial statements for numpy and pandas packages
import numpy as np
import pandas as pd
import csv
import seaborn as sns
import scipy as sp
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib as mpl 
from mpl_toolkits.mplot3d import Axes3D 


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
edudata = pd.read_csv('CH11PR08.txt', delimiter='\s+',
            names = ['Y (Curr. Salary)', 'Degree', 'X3 (Yrs Exp. Post-Ed)', 'X4 (Amt. Supervised)'])

#Let's see what a sample of 5 rows looks like to verify that data frame is as expected
edudata.sample(5)


,Y (Curr. Salary),Degree,X3 (Yrs Exp. Post-Ed),X4 (Amt. Supervised)
21,44.3,2,2.33,0
47,36.1,1,4.37,0
59,87.4,3,2.37,13
29,39.1,2,1.00,0
64,104.8,3,19.81,24


In [ ]:
#Now we add in two columns for X1 and X2 which are indicator variables as described earlier to denote
#bachelor's, master's, or doctoral degree
edudata['X1 (HigherEd1)'] = np.where(edudata['Degree']==2, 1, 0)
edudata.sample(5)
edudata['X2 (HigherEd2)'] = np.where(edudata['Degree']==3, 1, 0)
edudata.sample(5)

#index column
edudata['Index'] = list(range(len(edudata.index)))

b) Regress Y on X1, X2, X3, and X4, using a first-order model and ordinary least squares. Obtain the residuals, and plot them against Y ̂. What does the residual plot suggest?

In [ ]:
X = edudata[['X1 (HigherEd1)','X2 (HigherEd2)','X3 (Yrs Exp. Post-Ed)', 'X4 (Amt. Supervised)']] 
y = edudata['Y (Curr. Salary)']
## fit a OLS 
X = sm.add_constant(X)
edufit = sm.OLS(y, X).fit()
edufit.summary()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Y (Curr. Salary)   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.854
Method:                 Least Squares   F-statistic:                     94.76
Date:                Thu, 28 Nov 2019   Prob (F-statistic):           3.16e-25
Time:                        01:13:36   Log-Likelihood:                -240.23
No. Observations:                  65   AIC:                             490.5
Df Residuals:                      60   BIC:                             501.3
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    31.4714      2.869     10.969      0.000      25.732      37.210
X1 (HigherEd1)           10.8120      3.218      3.360      0.001       4.374      17.250
X2 (HigherEd2)           22.6307      3.485      6.494      0.000      15.660      29.601
X3 (Yrs Exp. Post-Ed)     1.2581      0.227      5.535      0.000       0.803       1.713
X4 (Amt. Supervised)      1.8523      0.228      8.137      0.000       1.397       2.308
==============================================================================
Omnibus:                       13.921   Durbin-Watson:                   2.122
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               29.803
Skew:                           0.587   Prob(JB):                     3.38e-07
Kurtosis:                       6.103   Cond. No.                         48.0
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
# fitted values (need a constant term for intercept)
edufit_fitted_y = edufit.fittedvalues
edudata['Yfit'] = edufit_fitted_y

edu_residuals = edufit.resid
edudata['Res'] = edu_residuals

#INFLUENCE OLS Influence results
eduinfluences = edufit.get_influence()

eduinfluences.summary_table
as_csv(eduinfluences)

# normalized residuals ri
edu_norm_residuals = edufit.get_influence().resid_studentized_internal

####dont know what this is
# absolute squared normalized residuals= sums to MSE?
edu_norm_residuals_abs_sqrt = np.sqrt(np.abs(edu_norm_residuals))

# absolute residuals
edu_abs_resid = np.abs(edu_residuals)
edudata['ABS RES'] = edu_abs_resid

# leverage, from statsmodels internals
edu_leverage = edufit.get_influence().hat_matrix_diag
edudata['hii'] = edu_leverage

########################################
# dfbetas, from statsmodels internals

#edudata['dfbetas'] = edu_dfbeta

# cook's distance, from statsmodels internals
edu_cooks = edufit.get_influence().cooks_distance[0]
edudata['Cooks distance'] = edu_cooks

#edudata.head(10)


NameError: ignored

In [ ]:
plt.style.use('seaborn') 
plt.rc('font', size=14)
plt.rc('figure', titlesize=18)
plt.rc('axes', labelsize=15)
plt.rc('axes', titlesize=18)

In [ ]:
resplot = sns.residplot(edudata['Index'], edudata['Res'], lowess=True, color="g")
resplot.axes.set_title('Residual Plot')
resplot.axes.set_xlabel(' Index')
resplot.axes.set_ylabel('Residuals')

In [ ]:
###########################################################
############ WORK IN PROGRESS- this is just residual plot not standardized
resplotst = sns.residplot(edudata['Index'], edudata['Res'], lowess=True, color="g")
resplotst.axes.set_title('Standardized Residual Plot')
resplotst.axes.set_xlabel('Index')
resplotst.axes.set_ylabel('Standardized Residuals')

In [ ]:
#partial regression plots
fig2 = plt.figure(figsize=(12,8))
fig2 = sm.graphics.plot_partregress_grid(edufit, fig=fig2)

In [ ]:
#Influence Plot

fig3, ax = plt.subplots(figsize=(8,6))
fig3 = sm.graphics.influence_plot(edufit, ax=ax)

c) Divide the cases into two groups, placing the 33 cases with the smallest fitted values Y ̂ into group 1 and the other 32 cases into group 2. Conduct the Brown-Forsythe test for constancy of the error variance, using α = .01. State the decision rule and conclusion.



In [ ]:
#I will create new dataframe sorted on Y fit values and split into the first group 33 cases and then second group 32 cases

edudata_1 = edudata.sort_values(by=['Yfit'])


#split into two groups
edudata_group1 = edudata_1.iloc[:33, :]
edudata_group2 = edudata_1.iloc[33:,: ]
print(edudata_group1.median())
print(edudata_group2.median())


#use Levene's test modified for ‘median’ (median is default; Recommended for skewed (non-normal) distributions)
brownforsythe = sp.stats.levene(edudata_group1['Y (Curr. Salary)'], edudata_group2['Y (Curr. Salary)'])
brownforsythe

d) Plot the absolute residuals against X3 and against X4. What do these plots suggest about the relation between the standard deviation of the error term and X3 and X4? 

In [ ]:
###########################################################
resplotvx3 = sns.residplot(edudata['X3 (Yrs Exp. Post-Ed)'], edudata['ABS RES'], lowess=True, color="c")
resplotvx3.axes.set_title('Absolute Residuals against X3')
resplotvx3.axes.set_xlabel('X3 (Yrs Exp. Post-Ed)')
resplotvx3.axes.set_ylabel('Absolute Residuals')



In [ ]:
###########################################################
resplotvx4 = sns.residplot(edudata['X4 (Amt. Supervised)'], edudata['ABS RES'], lowess=True, color="m")
resplotvx4.axes.set_title('Absolute Residuals against X3')
resplotvx4.axes.set_xlabel('X4 (Amt. Supervised)')
resplotvx4.axes.set_ylabel('Absolute Residuals')

e. Estimate the standard deviation function by regressing the absolute residuals against X3 and X4 in first-order form, and then calculate the estimated weight for each case using 11.16a.
11.16a: wi = 1/(s ̂i)^2 
Where s ̂i is the fitted value from the standard deviation function



In [ ]:
X = edudata[['X3 (Yrs Exp. Post-Ed)', 'X4 (Amt. Supervised)']] 
y = edudata['ABS RES']
## fit a OLS 
X = sm.add_constant(X)
edufit_res_34 = sm.OLS(y, X).fit()
edufit_res_34.summary()

In [ ]:
# fitted values (need a constant term for intercept)
edufit_fitted_s_34 = edufit_res_34.fittedvalues
edudata['s_34'] = edufit_fitted_s_34
edudata['w_34'] = 1/((edufit_fitted_s_34)**2)
print(edudata.loc[:,['s_34','w_34']])


f) Using the estimated weights, obtain the weighted least squares fit of the regression model. Are the weighted least squares estimates of the regression coefficients similar to the ones obtained with OLS in b)?

In [ ]:
X = edudata[['X1 (HigherEd1)','X2 (HigherEd2)','X3 (Yrs Exp. Post-Ed)', 'X4 (Amt. Supervised)']]
y = edudata['Y (Curr. Salary)']
## fit a WLS 
X = sm.add_constant(X)
edufit_WLS_34 = sm.WLS(y,X, weights= edudata['w_34']).fit()
edufit_WLS_34.summary()



g) Compare the estimated standard deviations of the weighted least squares coefficients estimates in part f) with those for the ordinary least squares in part b. What do you find?

In [ ]:
# abs residuals for weighted model
edu_residuals_1W = edufit_WLS_34.resid
edu_abs_resid_1W = np.abs(edu_residuals_1W)
edudata['ABS RES 1W'] = edu_abs_resid_1W

X = edudata[['X3 (Yrs Exp. Post-Ed)', 'X4 (Amt. Supervised)']] 
y = edudata['ABS RES 1W']
## fit a OLS 
X = sm.add_constant(X)
edufit_res_34_1W = sm.OLS(y, X).fit()
edufit_res_34_1W.summary()

edufit_fitted_s_34_1W= edufit_res_34_1W.fittedvalues
edudata['s_34_WLS'] = edufit_fitted_s_34_1W
edudata['w_34_WLS'] = 1/((edufit_fitted_s_34_1W)**2)
print(edudata.loc[:,['s_34','w_34', 's_34_WLS', 'w_34_WLS']])

In [ ]:
X = edudata[['X1 (HigherEd1)','X2 (HigherEd2)','X3 (Yrs Exp. Post-Ed)', 'X4 (Amt. Supervised)']]
y = edudata['Y (Curr. Salary)']
## fit a WLS 
X = sm.add_constant(X)
edufit_WLS_34_2 = sm.WLS(y,X, weights= edudata['w_34_WLS']).fit()
edufit_WLS_34_2.summary()

